In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import time

import numpy as np
try:
    import mxnet as mx
except:
    sys.stderr.write("Cannot import mxnet.\n")
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.ndimage.interpolation import map_coordinates
from skimage.exposure import rescale_intensity
from skimage.transform import rotate

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from metadata import *
from data_manager import *
from learning_utilities import *
from distributed_utilities import *
from visualization_utilities import *

/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
No vtk


Setting environment for Precision WorkStation


In [2]:
detector_settings = read_csv(DETECTOR_SETTINGS_CSV, header=0, index_col=0)

In [3]:
detector_settings

,input_version,windowing_id,feature_network,feature_classifier_id
detector_id,,,,
1,gray,1,inception-bn-blue,38
2,grayL500H0,1,inception-bn-blue,38
3,grayL1000H0,1,inception-bn-blue,38
4,grayL1500H0,1,inception-bn-blue,38
5,grayL2000H0,1,inception-bn-blue,38
6,grayL2500H0,1,inception-bn-blue,38
7,grayL600H0,1,inception-bn-blue,38
8,grayL700H0,1,inception-bn-blue,38
9,grayL800H0,1,inception-bn-blue,38


In [10]:
stack = 'MD661'
filenames = [fn for fn in metadata_cache['valid_filenames'][stack] if fn.split('-')[1][0] == 'F'][83:86]

In [ ]:
# for low in range(500, 2500, 500):
# for low in range(0, 1000, 100):
for low in [600,700,800,900]:
    
    script_fp = os.path.join(REPO_DIR, 'preprocess', 'convert_grayscale_neurotrace_v2.py')
    high = 0
    output_version = 'grayL'+str(low)+'H'+str(high)

    for fn in filenames:

        # try:
        t = time.time()
        img_fp = DataManager.get_image_filepath_v2(stack=stack, fn=fn, prep_id=2, resol='lossless')
        download_from_s3(img_fp, local_root=DATA_ROOTDIR)

        sys.stderr.write('Download: %.2f seconds\n' % (time.time() - t))

        if stack not in all_nissl_stacks and fn.split('-')[1][0] == 'F':
            # Neurotrace sections.

            t = time.time()
            img_blue = imread(img_fp)[..., 2]
            sys.stderr.write('Read: %.2f seconds\n' % (time.time() - t))

#             if not hasattr(args, 'low') or (hasattr(args, 'low') and args.low is None):
#                 sys.stderr.write("No linear limits arguments are given, so use nonlinear mapping.\n")

#                 if use_section_specific_mapping:
#                     try:
#                         intensity_mapping_fp = DataManager.get_ntb_to_nissl_intensity_profile_mapping_filepath(stack=stack, ntb_fn=fn)
#                         download_from_s3(intensity_mapping_fp)
#                         intensity_mapping_ntb_to_nissl = np.load(intensity_mapping_fp)
#                         load_default_mapping = False
#                         sys.stderr.write("Loaded section specific mapping.\n")
#                     except:
#                         sys.stderr.write("Error loading section-specific ntb-to-nissl intensity mapping. Load a default mapping instead.\n")
#                         load_default_mapping = True
#                 else:
#                     load_default_mapping = True

#                 if load_default_mapping:            
#                     intensity_mapping_fp = DataManager.get_ntb_to_nissl_intensity_profile_mapping_filepath()
#                     download_from_s3(intensity_mapping_fp)
#                     intensity_mapping_ntb_to_nissl = np.load(intensity_mapping_fp)

#                 t = time.time()
#                 img_blue_intensity_normalized = intensity_mapping_ntb_to_nissl[img_blue.astype(np.int)].astype(np.uint8)
#                 sys.stderr.write('Convert: %.2f seconds\n' % (time.time() - t))

#                 output_fp = DataManager.get_image_filepath_v2(stack=stack, fn=fn, prep_id=2, version=output_version, resol='lossless')

#             else:
            sys.stderr.write("Linear limits arguments detected, so use linear mapping.\n")

#             low_limit = args.low
#             high_limit = args.high

            t = time.time()
            img_blue_intensity_normalized = rescale_intensity_v2(img_blue, low, high)
            sys.stderr.write('Convert: %.2f seconds\n' % (time.time() - t))

            output_fp = DataManager.get_image_filepath_v2(stack=stack, fn=fn, prep_id=2, version=output_version, resol='lossless')

            t = time.time()
            create_parent_dir_if_not_exists(output_fp)
            imsave(output_fp, img_blue_intensity_normalized)
            sys.stderr.write('Save: %.2f seconds\n' % (time.time() - t))

            t = time.time()
            upload_to_s3(output_fp, local_root=DATA_ROOTDIR)
            sys.stderr.write('Upload: %.2f seconds\n' % (time.time() - t))

        else:
            # Nissl sections.

            t = time.time()
            img = imread(img_fp)
            sys.stderr.write('Read: %.2f seconds\n' % (time.time() - t))

            t = time.time()
            img_gray = img_as_ubyte(rgb2gray(img))
            sys.stderr.write('Convert: %.2f seconds\n' % (time.time() - t))

            output_fp = DataManager.get_image_filepath_v2(stack=stack, fn=fn, prep_id=2, version=output_version, resol='lossless')
            create_parent_dir_if_not_exists(output_fp)

            t = time.time()
            imsave(output_fp, img_gray)
            sys.stderr.write('Save: %.2f seconds\n' % (time.time() - t))

            t = time.time()
            upload_to_s3(output_fp, local_root=DATA_ROOTDIR)
            sys.stderr.write('Upload: %.2f seconds\n' % (time.time() - t))

In [6]:
############

model_dir_name = 'inception-bn-blue'
download_from_s3(os.path.join(MXNET_MODEL_ROOTDIR, model_dir_name), is_dir=True)
model_name = 'inception-bn-blue'
model_iteration = 0
output_symbol_name = 'flatten_output'
output_dim = 1024
mean_img = np.load(os.path.join(MXNET_MODEL_ROOTDIR, model_dir_name, 'mean_224.npy'))

# Reference on how to predict with mxnet model:
# https://github.com/dmlc/mxnet-notebooks/blob/master/python/how_to/predict.ipynb
model0, arg_params, aux_params = mx.model.load_checkpoint(os.path.join(MXNET_MODEL_ROOTDIR, model_dir_name, model_name), 0)
flatten_output = model0.get_internals()[output_symbol_name]
if HOST_ID == 'workstation':
    model = mx.mod.Module(context=[mx.gpu(i) for i in range(1)], symbol=flatten_output)
else:
    model = mx.mod.Module(context=[mx.gpu(i) for i in range(8)], symbol=flatten_output)

# Increase batch_size to 500 does not save any time.
batch_size = 256
model.bind(data_shapes=[('data', (batch_size,1,224,224))], for_training=False)
model.set_params(arg_params=arg_params, aux_params=aux_params, allow_missing=True)

###############

/usr/local/lib/python2.7/dist-packages/mxnet/module/base_module.py:52: UserWarning: You created Module with Module(..., label_names=['softmax_label']) but input with name 'softmax_label' is not found in symbol.list_arguments(). Did you mean one of:
	data
  warnings.warn(msg)
/usr/local/lib/python2.7/dist-packages/mxnet/module/base_module.py:64: UserWarning: Data provided by label_shapes don't match names specified by label_names ([] vs. ['softmax_label'])
  warnings.warn(msg)


In [11]:
stack = 'MD661'
section_to_filename = metadata_cache['sections_to_filenames'][stack]

windowing_id = 1

windowing_properties = windowing_settings[windowing_id]
patch_size = windowing_properties['patch_size']
spacing = windowing_properties['spacing']
w, h = metadata_cache['image_shape'][stack]
half_size = patch_size/2

grid_spec = (patch_size, spacing, w, h)

sample_locations = grid_parameters_to_sample_locations(grid_spec=grid_spec)

In [19]:
for low in [600,700,800,900]:
    
    high = 0
    input_img_version = 'grayL'+str(low)+'H'+str(high)

    for fn in filenames:

        t = time.time()
        mask_tb = DataManager.load_thumbnail_mask_v3(stack=stack, prep_id=2, fn=fn)
        indices_roi = locate_patches_v2(grid_spec=grid_spec, mask_tb=mask_tb)
        sys.stderr.write('locate patches: %.2f seconds\n' % (time.time() - t))       

        n = len(indices_roi)
        sys.stderr.write('%d samples.\n' % n)

        ############################

        sample_locations_roi = sample_locations[indices_roi]

        ###########################

        t = time.time()
        im = DataManager.load_image_v2(stack=stack, fn=fn, prep_id=2, 
                                       version=input_img_version, resol='lossless')
        sys.stderr.write('load image: %.2f seconds\n' % (time.time() - t)) # ~ 35s

        patches = np.array([im[y-half_size:y+half_size, x-half_size:x+half_size]
                            for x, y in sample_locations_roi]) # n x 224 x 224

        #############################

        patches_mean_subtracted = patches - mean_img
        patches_mean_subtracted_input = patches_mean_subtracted[:, None, :, :] # n x 1 x 224 x 224

        t1 = time.time()        
        data_iter = mx.io.NDArrayIter(
                        patches_mean_subtracted_input, 
                        batch_size=batch_size,
                        shuffle=False)
        outputs = model.predict(data_iter, always_output_list=True)
        features = outputs[0].asnumpy()

        del data_iter

        sys.stderr.write('predict: %.2f seconds\n' % (time.time() - t1))

        ################################

        t = time.time()

        features_fp = DataManager.get_dnn_features_filepath(stack=stack, model_name=model_name, 
                                                            win=windowing_id,
                                                            fn=fn,
                                                           input_img_version=input_img_version)
        create_parent_dir_if_not_exists(features_fp)
        bp.pack_ndarray_file(features.astype(np.float16), features_fp) # bloscpack produces files of similar size
        upload_to_s3(features_fp, local_root=DATA_ROOTDIR)

        del features

    #     feature_locs_fp = DataManager.get_patch_locations_filepath(stack=stack,
    #                                                                      win=windowing_id,
    #                                                                      section=sec)
    #     create_parent_dir_if_not_exists(feature_locs_fp)
    #     np.savetxt(feature_locs_fp, np.c_[indices_roi, sample_locations_roi], fmt='%d %d %d')
    #     upload_to_s3(feature_locs_fp, local_root=DATA_ROOTDIR)

        sys.stderr.write('save: %.2f seconds\n' % (time.time() - t)) # ~.5s

locate patches: 0.04 seconds
58197 samples.
load image: 2.37 seconds
predict: 100.58 seconds


aws s3 cp "/media/yuncong/BstemAtlasData/CSHL_patch_features/inception-bn-blue/MD661/MD661_prep2_grayL600H0_win1/MD662&661-F57-2017.06.06-02.06.22_MD661_3_0171_prep2_grayL600H0_win1_inception-bn-blue_features.bp" "s3://mousebrainatlas-data/CSHL_patch_features/inception-bn-blue/MD661/MD661_prep2_grayL600H0_win1/MD662&661-F57-2017.06.06-02.06.22_MD661_3_0171_prep2_grayL600H0_win1_inception-bn-blue_features.bp"


save: 5.10 seconds
locate patches: 0.05 seconds
58185 samples.
load image: 0.33 seconds
predict: 105.86 seconds


aws s3 cp "/media/yuncong/BstemAtlasData/CSHL_patch_features/inception-bn-blue/MD661/MD661_prep2_grayL600H0_win1/MD662&661-F58-2017.06.06-02.35.45_MD661_1_0172_prep2_grayL600H0_win1_inception-bn-blue_features.bp" "s3://mousebrainatlas-data/CSHL_patch_features/inception-bn-blue/MD661/MD661_prep2_grayL600H0_win1/MD662&661-F58-2017.06.06-02.35.45_MD661_1_0172_prep2_grayL600H0_win1_inception-bn-blue_features.bp"


save: 4.87 seconds
locate patches: 0.06 seconds
58403 samples.
load image: 0.31 seconds
predict: 111.83 seconds


...

save: 5.00 seconds
locate patches: 0.04 seconds
58185 samples.
load image: 0.31 seconds
predict: 109.57 seconds


aws s3 cp "/media/yuncong/BstemAtlasData/CSHL_patch_features/inception-bn-blue/MD661/MD661_prep2_grayL900H0_win1/MD662&661-F58-2017.06.06-02.35.45_MD661_1_0172_prep2_grayL900H0_win1_inception-bn-blue_features.bp" "s3://mousebrainatlas-data/CSHL_patch_features/inception-bn-blue/MD661/MD661_prep2_grayL900H0_win1/MD662&661-F58-2017.06.06-02.35.45_MD661_1_0172_prep2_grayL900H0_win1_inception-bn-blue_features.bp"


save: 5.53 seconds
locate patches: 0.04 seconds
58403 samples.
load image: 0.30 seconds
predict: 112.47 seconds


aws s3 cp "/media/yuncong/BstemAtlasData/CSHL_patch_features/inception-bn-blue/MD661/MD661_prep2_grayL900H0_win1/MD662&661-F58-2017.06.06-02.35.45_MD661_2_0173_prep2_grayL900H0_win1_inception-bn-blue_features.bp" "s3://mousebrainatlas-data/CSHL_patch_features/inception-bn-blue/MD661/MD661_prep2_grayL900H0_win1/MD662&661-F58-2017.06.06-02.35.45_MD661_2_0173_prep2_grayL900H0_win1_inception-bn-blue_features.bp"


save: 5.03 seconds


In [5]:
# This can also be done in `pipeline_aws.ipynb`.

for detector_id in [7,8,9,10]:           

    ###################################
    
    detector_properties = detector_settings.loc[detector_id]
    input_img_version = detector_properties['input_version']
    windowing_id = int(detector_properties['windowing_id'])
    model_name = detector_properties['feature_network']
    classifier_id = int(detector_properties['feature_classifier_id'])

    classifier_properties = classifier_settings.loc[classifier_id]
    assert dataset_settings.loc[int(classifier_properties['train_set_id'].split('/')[0])]['network_model'] == model_name
    svm_id = int(classifier_properties['svm_id'])

    windowing_properties = windowing_settings[windowing_id]
    patch_size = windowing_properties['patch_size']
    spacing = windowing_properties['spacing']
    w, h = metadata_cache['image_shape'][stack]
    half_size = patch_size/2
    
    ########################
    
    available_classifiers = {svm_id: DataManager.load_classifiers(classifier_id=svm_id)}

    def clf_predict(stack, fn):

        if is_invalid(stack=stack, fn=fn):
            return

        try:
            features = DataManager.load_dnn_features(stack=stack, model_name=model_name, fn=fn, 
                                                     win=windowing_id, input_img_version=input_img_version)
        except Exception as e:
            sys.stderr.write('%s\n' % e.message)
            return

        clf_allClasses_ = available_classifiers[svm_id]

        for structure, clf in clf_allClasses_.iteritems():

            probs = clf.predict_proba(features)[:, clf.classes_.tolist().index(1.)]

            output_fn = DataManager.get_sparse_scores_filepath(stack=stack, structure=structure, 
                                                               detector_id=detector_id, fn=fn)
            create_parent_dir_if_not_exists(output_fn)
            bp.pack_ndarray_file(probs, output_fn)
            upload_to_s3(output_fn)


    t = time.time()

    pool = Pool(NUM_CORES/2)
    pool.map(lambda fn: clf_predict(stack=stack, fn=fn), filenames)
    pool.close()
    pool.join()

    sys.stderr.write('Classifier predict: %.2f\n' % (time.time()-t))

    #############################

    downscale = 32

    ##############################

    def resample(fn):

        if is_invalid(stack=stack, fn=fn):
            return

    #         t = time.time()

        try:
            _, sample_locations_roi = DataManager.load_patch_locations(stack=stack, fn=fn, win=windowing_id)
        except Exception as e:
            sys.stderr.write('Error loading patch locations for %s: %s.\n' % (fn, str(e)))
            return

        #actual_setting = resolve_actual_setting(setting=classifier_id, stack=stack, fn=fn)

        downscaled_grid_y = np.arange(0, h, downscale)
        downscaled_grid_x = np.arange(0, w, downscale)
        downscaled_ny = len(downscaled_grid_y)
        downscaled_nx = len(downscaled_grid_x)

        for structure in all_structures_with_classifiers:
            try:
                sparse_scores = DataManager.load_sparse_scores(stack, fn=fn, structure=structure, detector_id=detector_id)
            except Exception as e:
                sys.stderr.write('Error loading sparse scores for %s: %s\n' % (structure, str(e)))
                continue

            f_grid = np.zeros(((h-half_size)/spacing+1, (w-half_size)/spacing+1))
            a = (sample_locations_roi - half_size)/spacing
            f_grid[a[:,1], a[:,0]] = sparse_scores

            yinterps = (downscaled_grid_y - half_size)/float(spacing)
            xinterps = (downscaled_grid_x - half_size)/float(spacing)

            points_y, points_x = np.broadcast_arrays(yinterps.reshape(-1,1), xinterps)
            coord = np.c_[points_y.flat, points_x.flat]
            f_interp = map_coordinates(f_grid, coord.T, order=1)
            f_interp_2d = f_interp.reshape((downscaled_ny, downscaled_nx))                       

            scoremap_bp_filepath = \
            DataManager.get_downscaled_scoremap_filepath(stack=stack, fn=fn, 
                                                         structure=structure, 
                                                         detector_id=detector_id,
                                                        downscale=downscale)

            create_parent_dir_if_not_exists(scoremap_bp_filepath)
            bp.pack_ndarray_file(f_interp_2d.astype(np.float16), scoremap_bp_filepath)
            upload_to_s3(scoremap_bp_filepath)

    #         sys.stderr.write('interpolate %d: %.2f seconds\n' % (sec, time.time() - t)) 

    t = time.time()

    pool = Pool(NUM_CORES/2)
    # pool = Pool(1)
    pool.map(resample, filenames)
    pool.close()
    pool.join()

    sys.stderr.write('Resample: %.2f seconds\n' % (time.time() - t)) 

    ###########################

    add_label_text = True
    cmap_name = 'jet'
    bg_image_version = input_img_version
    downscale = 8

    #########################

    for fn in filenames:

        if is_invalid(stack=stack, fn=fn):
            continue

        t = time.time()

        def save_scoremap(structure):
            viz_fp = DataManager.get_scoremap_viz_filepath(stack=stack, downscale=downscale, fn=fn, structure=structure, detector_id=detector_id)
            create_parent_dir_if_not_exists(viz_fp)

            try:
                if add_label_text:
                    label_text = str(structure)
                else:
                    label_text = None
                viz = scoremap_overlay_on(bg='original', stack=stack, fn=fn, structure=structure,
                                    out_downscale=downscale, label_text=label_text, detector_id=detector_id,
                                         cmap_name=cmap_name, image_version=bg_image_version)
                imsave(viz_fp, img_as_ubyte(viz))
                upload_to_s3(viz_fp)
            except Exception as e:
                # raise e
                sys.stderr.write('%s\n' % e.message)
                return

        # for s in all_known_structures:
            # save_scoremap(s)

        pool = Pool(NUM_CORES)
        pool.map(save_scoremap, all_known_structures)
        pool.close()
        pool.join()

        sys.stderr.write('Visualize scoremaps: %.2f seconds.\n' % (time.time() - t))
        # 7s for one structure, one section, single process
        # 20s for all structures, one section, 8 processes


rm -rf "/home/yuncong/CSHL_classifiers/setting_38/classifiers/sp5_clf_setting_38.dump" && mkdir -p "/home/yuncong/CSHL_classifiers/setting_38/classifiers"
aws s3 cp "s3://mousebrainatlas-data/CSHL_classifiers/setting_38/classifiers/sp5_clf_setting_38.dump" "/home/yuncong/CSHL_classifiers/setting_38/classifiers/sp5_clf_setting_38.dump"


Setting 38: No classifier found for sp5.


rm -rf "/home/yuncong/CSHL_classifiers/setting_38/classifiers/outerContour_clf_setting_38.dump" && mkdir -p "/home/yuncong/CSHL_classifiers/setting_38/classifiers"
aws s3 cp "s3://mousebrainatlas-data/CSHL_classifiers/setting_38/classifiers/outerContour_clf_setting_38.dump" "/home/yuncong/CSHL_classifiers/setting_38/classifiers/outerContour_clf_setting_38.dump"


Setting 38: No classifier found for outerContour.


rm -rf "/home/yuncong/media/yuncong/BstemAtlasData/CSHL_patch_features/inception-bn-blue/MD661/MD661_prep2_grayL600H0_win1/MD662&661-F58-2017.06.06-02.35.45_MD661_2_0173_prep2_grayL600H0_win1_inception-bn-blue_features.bp" && mkdir -p "/home/yuncong/media/yuncong/BstemAtlasData/CSHL_patch_features/inception-bn-blue/MD661/MD661_prep2_grayL600H0_win1"
rm -rf "/home/yuncong/media/yuncong/BstemAtlasData/CSHL_patch_features/inception-bn-blue/MD661/MD661_prep2_grayL600H0_win1/MD662&661-F58-2017.06.06-02.35.45_MD661_1_0172_prep2_grayL600H0_win1_inception-bn-blue_features.bp" && mkdir -p "/home/yuncong/media/yuncong/BstemAtlasData/CSHL_patch_features/inception-bn-blue/MD661/MD661_prep2_grayL600H0_win1"
rm -rf "/home/yuncong/media/yuncong/BstemAtlasData/CSHL_patch_features/inception-bn-blue/MD661/MD661_prep2_grayL600H0_win1/MD662&661-F57-2017.06.06-02.06.22_MD661_3_0171_prep2_grayL600H0_win1_inception-bn-blue_features.bp" && mkdir -p "/home/yuncong/media/yuncong/BstemAtlasData/CSHL_patch_featur

...

rm -rf "/home/yuncong/CSHL_patch_scores/MD661/MD662&661-F57-2017.06.06-02.06.22_MD661_3_0171_prep2_gray/detector10/MD662&661-F57-2017.06.06-02.06.22_MD661_3_0171_prep2_gray_detector10_VLL_sparseScores.bp" && mkdir -p "/home/yuncong/CSHL_patch_scores/MD661/MD662&661-F57-2017.06.06-02.06.22_MD661_3_0171_prep2_gray/detector10"
aws s3 cp "s3://mousebrainatlas-data/CSHL_patch_scores/MD661/MD662&661-F57-2017.06.06-02.06.22_MD661_3_0171_prep2_gray/detector10/MD662&661-F57-2017.06.06-02.06.22_MD661_3_0171_prep2_gray_detector10_VLL_sparseScores.bp" "/home/yuncong/CSHL_patch_scores/MD661/MD662&661-F57-2017.06.06-02.06.22_MD661_3_0171_prep2_gray/detector10/MD662&661-F57-2017.06.06-02.06.22_MD661_3_0171_prep2_gray_detector10_VLL_sparseScores.bp"


File does not exist: /home/yuncong/CSHL_patch_scores/MD661/MD662&661-F58-2017.06.06-02.35.45_MD661_1_0172_prep2_gray/detector10/MD662&661-F58-2017.06.06-02.35.45_MD661_1_0172_prep2_gray_detector10_VCP_sparseScores.bp
Error loading sparse scores for VCP: [Errno 2] No such file or directory: '/home/yuncong/CSHL_patch_scores/MD661/MD662&661-F58-2017.06.06-02.35.45_MD661_1_0172_prep2_gray/detector10/MD662&661-F58-2017.06.06-02.35.45_MD661_1_0172_prep2_gray_detector10_VCP_sparseScores.bp'


rm -rf "/home/yuncong/CSHL_patch_scores/MD661/MD662&661-F58-2017.06.06-02.35.45_MD661_1_0172_prep2_gray/detector10/MD662&661-F58-2017.06.06-02.35.45_MD661_1_0172_prep2_gray_detector10_VLL_sparseScores.bp" && mkdir -p "/home/yuncong/CSHL_patch_scores/MD661/MD662&661-F58-2017.06.06-02.35.45_MD661_1_0172_prep2_gray/detector10"
aws s3 cp "s3://mousebrainatlas-data/CSHL_patch_scores/MD661/MD662&661-F58-2017.06.06-02.35.45_MD661_1_0172_prep2_gray/detector10/MD662&661-F58-2017.06.06-02.35.45_MD661_1_0172_prep2_gray_detector10_VLL_sparseScores.bp" "/home/yuncong/CSHL_patch_scores/MD661/MD662&661-F58-2017.06.06-02.35.45_MD661_1_0172_prep2_gray/detector10/MD662&661-F58-2017.06.06-02.35.45_MD661_1_0172_prep2_gray_detector10_VLL_sparseScores.bp"


File does not exist: /home/yuncong/CSHL_patch_scores/MD661/MD662&661-F58-2017.06.06-02.35.45_MD661_2_0173_prep2_gray/detector10/MD662&661-F58-2017.06.06-02.35.45_MD661_2_0173_prep2_gray_detector10_VLL_sparseScores.bp
Error loading sparse scores for VLL: [Errno 2] No such file or directory: '/home/yuncong/CSHL_patch_scores/MD661/MD662&661-F58-2017.06.06-02.35.45_MD661_2_0173_prep2_gray/detector10/MD662&661-F58-2017.06.06-02.35.45_MD661_2_0173_prep2_gray_detector10_VLL_sparseScores.bp'
File does not exist: /home/yuncong/CSHL_patch_scores/MD661/MD662&661-F57-2017.06.06-02.06.22_MD661_3_0171_prep2_gray/detector10/MD662&661-F57-2017.06.06-02.06.22_MD661_3_0171_prep2_gray_detector10_VLL_sparseScores.bp
Error loading sparse scores for VLL: [Errno 2] No such file or directory: '/home/yuncong/CSHL_patch_scores/MD661/MD662&661-F57-2017.06.06-02.06.22_MD661_3_0171_prep2_gray/detector10/MD662&661-F57-2017.06.06-02.06.22_MD661_3_0171_prep2_gray_detector10_VLL_sparseScores.bp'
File does not exist: /

'str' object has no attribute 'shape'
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:157: DeprecationWarning: BaseException.message has been deprecated as of Python 2.6
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:157: DeprecationWarning: BaseException.message has been deprecated as of Python 2.6
'str' object has no attribute 'shape'
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:157: DeprecationWarning: BaseException.message has been deprecated as of Python 2.6
'str' object has no attribute 'shape'
'str' object has no attribute 'shape'
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:157: DeprecationWarning: BaseException.message has been deprecated as of Python 2.6
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:157: DeprecationWarning: BaseException.message has been deprecated as of Python 2.6
'str' object has no attribute 'shape'
'str' object has no attribute 'shape'
'str' object has no attribute 'shape'
/usr/loca